In [1]:
data_path = '../datasets/AICity_data/train/S03/c010/'

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm
from PIL import Image
from eval_utils import *
from video_utils import *
from load_utils import *
from background_remover import *
from lobster import Subsense

In [3]:
def mIoU_calc(IoU):
    mIoU = 0
    mIoUPerFrame = []
    count = 0
    for fr, values in IoU.items():
        mIoU += np.sum(values)
        count += len(values)
        mIoUPerFrame.append(np.mean(values))
    mIoU /= count
    
    return mIoU

In [4]:
def drawRectangleOnImage(img, boxes, color=(0, 255, 0)):
    for box in boxes:
        img = cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 3)
    return img

In [5]:
def getBoxesFromMaskV2(mask):
    output = cv2.connectedComponentsWithStats(np.uint8(mask), 8, cv2.CV_32S)
    (numLabels, labels, boxes, centroids) = output
    detectedElems = []
    for box in boxes[1:]: #First box is always the background
        if box[4] > 500: #Try to do a better estimation of the minimunm size
            b = [box[0], box[1], box[2], box[3]]
            detectedElems.append(b)

    return detectedElems

In [6]:
#############TEST################
# cap = cv2.VideoCapture(data_path + 'vdo.avi')
  
# # initializing subtractor 
# fgbg = cv2.bgsegm.createBackgroundSubtractorMOG() 
  
# while(1):
#     ret, frame = cap.read()       
  
#     # applying on each frame
#     fgmask = fgbg.apply(frame)
#     fgmask = cleanMask(fgmask, 5)
#     boxes = getBoxesFromMaskV2(fgmask)
#     fgmask = drawRectangleOnImage(fgmask, boxes)
  
#     cv2.imshow('frame', fgmask)
#     k = cv2.waitKey(10) & 0xff
#     if k == 27:
#         break
  
# cap.release()
# cv2.destroyAllWindows()

In [7]:
def generate_gif(videoPath, fgbg, videoName='MOG2'):
    fig, ax = plt.subplots()
    plt.axis('off')
    
    vidcap = cv2.VideoCapture(videoPath)
#     _, image = vidcap.read()
    ims = []
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
#     num_frames = 400

    for frame in tqdm(range(1,num_frames//4)):
        for i in range(4):
            _, image = vidcap.read()
        fgmask = fgbg.apply(image)
        fgmask = cleanMask(fgmask, 5)
        
        im = ax.imshow(fgmask, animated=True)
        ims.append([im])
    # break

    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
    ani.save(videoName + ".gif", writer=animation.PillowWriter(fps=24))

In [8]:
# fgbg = cv2.bgsegm.createBackgroundSubtractorMOG() 

In [9]:
# fgbg = cv2.createBackgroundSubtractorMOG2()

In [10]:
# fgbg = Subsense()

OSError: no file with expected extension

In [ ]:
generate_gif(data_path + 'vdo.avi', fgbg, 'test')

In [ ]:
print(f'mIoU: {mIoU_calc(IoU)}')

#Calculating AP:
print(f'mAP: {ap}')

plot_prec_recall_curve(prec, rec, 'Precision-Recall curve for noised detections')

In [ ]:
# gt_detect = readDetections(data_path + 'gt/gt.txt')
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')